In [ ]:
import networkit as nk
import networkit.vizbridges
import ipywidgets as iw
import plotly

def apply_layout(widget):
    widget.update_layout(margin=plotly.graph_objs.layout.Margin(l=10, r=10, b=0, t=0))

G = nk.readGraph("input/karate.graph", nk.Format.METIS)

# New Feature: 3D Plots

In [ ]:
def make_widget(algo):
    if algo == 'Electrical Closeness':
        centrality = nk.centrality.ApproxElectricalCloseness(G)
    elif algo == 'Harmonic':
        centrality = nk.centrality.HarmonicCloseness(G)
    else:
        raise RuntimeError('Bad algorithm')
    centrality.run()
    widget = nk.vizbridges.widgetFromGraph(G, dimension = nk.vizbridges.Dimension.Three,
                                         nodeScores=centrality.scores(), customSize=500)
    apply_layout(widget)
    return widget

iw.interact(make_widget, algo=['Electrical Closeness', 'Harmonic']);

# New Algorithms: (Parallel) Leiden, Louvain based on Map Equation

In [ ]:
def make_widget(algo):
    if algo == 'Leiden':
        community = nk.community.ParallelLeiden(G)
        community.run()
        p = community.getPartition()
        p.compact()
    elif algo == 'Map Equation':
        community = nk.community.LouvainMapEquation(G)
        community.run()
        p = community.getPartition()
        p.compact()
    else:
        raise RuntimeError('Bad algorithm')

    widget = nk.vizbridges.widgetFromGraph(G, dimension = nk.vizbridges.Dimension.Three,
                                           nodePartition=p, customSize=500)
    apply_layout(widget)
    return widget

iw.interact(make_widget, algo=['Leiden', 'Map Equation']);

# New Algorithm: Local Fitness Method (LFM)

In [ ]:
local = nk.scd.LFMLocal(G)
lfm = nk.community.LFM(G, local)
lfm.run()
cover = lfm.getCover()

parts = list(range(0, cover.numberOfSubsets()))

def make_widget(part):
    s = set(cover.getMembers(part))
    p = nk.Partition(G.upperNodeIdBound())
    p.allToSingletons()
    for u in range(0, G.upperNodeIdBound()):
        p.moveToSubset(1 if u in s else 0, u)

    widget = nk.vizbridges.widgetFromGraph(G, dimension = nk.vizbridges.Dimension.Three,
                                           nodePartition=p, customSize=500)
    apply_layout(widget)
    return widget

iw.interact(make_widget, part=parts);